### Create a UC Function-Tool
In this notebook we will create a python function and we will save it into Unity Catalog. After we will test it 

In [0]:
# Install Unity Catalog AI integration packages with the Databricks extra
%pip install unitycatalog-ai[databricks]
%pip install unitycatalog-langchain[databricks]
%pip install -U -qqqq mlflow databricks-openai databricks-agents

# Install the Databricks LangChain integration package
%pip install databricks-langchain


dbutils.library.restartPython()

In [0]:
import sys
path = '/Workspace' + '/'.join(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get().split("/")[0:-2])
sys.path.append(path)

In [0]:
from configs import variables

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

In [0]:
def add_numbers(number_1: float, number_2: float) -> float:
  """
  A function that accepts two floating point numbers adds them,
  and returns the resulting sum as a float.

  Args:
    number_1 (float): The first of the two numbers to add.
    number_2 (float): The second of the two numbers to add.

  Returns:
    float: The sum of the two input numbers.
  """
  return number_1 + number_2

function_info = client.create_python_function(
  func=add_numbers,
  catalog=variables.CATALOG_NAME,
  schema=variables.SCHEMA_NAME,
  replace=True
)

In [0]:
def subtract_numbers(number_1: float, number_2: float) -> float:
    """
    A function that subtracts one number from another.

    Args:
        number_1 (float): The number from which to subtract.
        number_2 (float): The number to subtract.

    Returns:
        float: The result of the subtraction.
    """
    return number_1 - number_2

function_info = client.create_python_function(
  func=subtract_numbers,
  catalog=variables.CATALOG_NAME,
  schema=variables.SCHEMA_NAME,
  replace=True
)

In [0]:
def multiply_numbers(number_1: float, number_2: float) -> float:
    """
    A function that multiplies two numbers.

    Args:
        number_1 (float): The first of the two numbers to multiply.
        number_2 (float): The second of the two numbers to multiply.

    Returns:
        float: The product of the two input numbers.
    """
    return number_1 * number_2

function_info = client.create_python_function(
  func=multiply_numbers,
  catalog=variables.CATALOG_NAME,
  schema=variables.SCHEMA_NAME,
  replace=True
)

In [0]:
def divide_numbers(number_1: float, number_2: float) -> float:
    """
    A function that divides one number by another.

    Args:
        number_1 (float): The numerator.
        number_2 (float): The denominator.

    Returns:
        float: The quotient of the two input numbers.

    Raises:
        ValueError: If division by zero is attempted.
    """
    if number_2 == 0:
        raise ValueError("Division by zero is not allowed.")
    return number_1 / number_2

function_info = client.create_python_function(
  func=divide_numbers,
  catalog=variables.CATALOG_NAME,
  schema=variables.SCHEMA_NAME,
  replace=True
)

In [0]:
def average_numbers(numbers: list[float]) -> float:
    """
    A function that calculates the average of a list of numbers.

    Args:
        numbers (list[float]): A list of numeric values.

    Returns:
        float: The average of the provided numbers.

    Raises:
        ValueError: If the list of numbers is empty.
    """
    if not numbers:
        raise ValueError("List of numbers cannot be empty.")
    return sum(numbers) / len(numbers)

function_info = client.create_python_function(
  func=average_numbers,
  catalog=variables.CATALOG_NAME,
  schema=variables.SCHEMA_NAME,
  replace=True
)

In [0]:
%sql
CREATE OR REPLACE FUNCTION agentic_ai.synthia_data_agent.list_catalogs() 
RETURNS ARRAY<STRING> 
COMMENT 'Returns a list of available catalog names'
RETURN 
SELECT COLLECT_LIST(catalog_name) 
FROM system.information_schema.catalogs
WHERE catalog_name NOT IN ('information_schema', 'system')

### Test Function

In [0]:
result = client.execute_function(
  function_name=f"{variables.CATALOG_NAME}.{variables.SCHEMA_NAME}.list_catalogs",
  parameters={}
)

result.value

In [ ]:
print("Available catalogs:")
result = client.execute_function(
    function_name=f"{variables.CATALOG_NAME}.{variables.SCHEMA_NAME}.list_catalogs",
    parameters={}
)

# Print the list of catalog names
for catalog_name in result.value:
    print(f"- {catalog_name}")

In [0]:
result

###  Agent Tool
Wrap the function using the UCFunctionToolkit to make it accessible to agent authoring libraries. The toolkit ensures consistency across different gen AI libraries and adds helpful features like auto-tracing for retrievers.

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from databricks_langchain import (
  ChatDatabricks,
  UCFunctionToolkit,
)
import mlflow

# Initialize the LLM (optional: replace with your LLM of choice)
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.1)

# Define the prompt
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Make sure to use tools for additional functionality.",
    ),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
  ]
)

# Enable automatic tracing
mlflow.langchain.autolog()

# Define the agent, specifying the tools from the toolkit above
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is 36939.0 + 8922.4?"})